This notebook is used to reproduce the Figure 5 in the paper.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ortools_solver import CVRP_solver
from utils import random_task_generation
from pipette_scheduler import calculate_D, calculate_S_E, calculate_D_prime,calculate_X
from baseline_methods import LAP, greedy
import pandas as pd

In [2]:
# define the types of labware
labware_list =[12,24,96,384]
# enumerate all the two combinations of the labware_list
labware_combinations = []
for i in range(len(labware_list)):
    for j in range(len(labware_list)):
        labware_combinations.append([labware_list[i], labware_list[j]])


In [3]:
# establish an empty dataframe to store the results
df = pd.DataFrame(columns=['source_labware', 'dest_labware', 'num_samples', 'repeat', 'row-major sorting','LAP', 'greedy','CVRP'])

In [ ]:
np.random.seed(0)
for labware_combination in labware_combinations: 

    source_dim = labware_combination[0]
    dest_dim = labware_combination[1]
    print(f'source_dim={source_dim}, dest_dim={dest_dim}')
    # repeat the experiment 3 times
    for r in range(3):
        print('repeat:',r+1)
        stats = []
        for i in range(1, 11, 1):

            num_candidates = dest_dim * i 
            num_candidates = int(num_candidates)
            print(f'num_candidates={num_candidates}')
            a = random_task_generation(source_dim,dest_dim,num_candidates)
            jobs = np.argwhere(a)
            D_S = calculate_D(a.shape[0])
            D_D = calculate_D(a.shape[1])
            S, E, volumes = calculate_S_E(a)

            # calculate distance matrix for CVRP
            D_prime = calculate_D_prime(D_S, D_D, S, E, volumes, 1, 100, 1, 100)
            # solve the CVRP problem, the D_prime should be multiplied by 100 to avoid floating point issues
            CVRP_distance, CVRP_recorder = CVRP_solver(np.round(D_prime * 100).astype(np.int64), solving_time=20)
            # bring the VRP_distance to the original scale
            CVRP_distance = CVRP_distance / 100
            print(f'CVRP_distance: {CVRP_distance}')

            # calculate the cost of the row-major sorting
            # create a sequence of tasks
            tasks = np.array(range(jobs.shape[0]))
            tasks = tasks+1
            # if tasks.shape[0] %8 != 0, pad with -1
            if tasks.shape[0] %8 != 0:
                tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
            rms_seuqnece = tasks.reshape(-1, 8)
            t = calculate_X(rms_seuqnece)
            d = np.round(D_prime * 100)
            rms_distance = np.trace(np.dot(t.T, d))/100
            rms_distance = int(rms_distance)
            print(f'row-major sorting: {rms_distance}')
            
            # calculate the cost of the LAP
            index_matrix = np.zeros((source_dim,dest_dim))
            for j in range(jobs.shape[0]):
                index_matrix[jobs[j, 0], jobs[j, 1]] = j+1
            LAP_sequence = LAP(index_matrix)
            if LAP_sequence.shape[0] %8 != 0:
                LAP_sequence = np.pad(LAP_sequence, (0, 8-LAP_sequence.shape[0]%8), 'constant', constant_values=-1)
            LAP_sequence = LAP_sequence.reshape(-1, 8)
            t = calculate_X(LAP_sequence)
            LAP_distance = np.trace(np.dot(t.T, d))/100
            # change non_optimized_distance to integer
            LAP_distance = int(LAP_distance)
            print(f'LAP_distance: {LAP_distance}')

            # calculate the cost of the greedy optimized sequence
            greedy_optimized_sequence = greedy(jobs, d[1:,1:])
            if greedy_optimized_sequence.shape[0] %8 != 0:
                greedy_optimized_sequence = np.pad(greedy_optimized_sequence, (0, 8-greedy_optimized_sequence.shape[0]%8), 'constant', constant_values=-1)
            greedy_optimized_sequence = greedy_optimized_sequence.reshape(-1, 8)
            t = calculate_X(greedy_optimized_sequence)
            greedy_optimized_distance = np.trace(np.dot(t.T, d))/100
            print(f'greedy_optimized_distance: {greedy_optimized_distance}')
            # append the results to the df
            stats.append([source_dim, dest_dim, num_candidates, r+1, rms_distance, LAP_distance, greedy_optimized_distance, CVRP_distance])
        # convert the stats to the dataframe
        stats = pd.DataFrame(stats, columns=['source_labware', 'dest_labware', 'num_samples', 'repeat', 'row-major sorting','LAP', 'greedy','CVRP'])
        # append the stats to the df
        df = pd.concat([df, stats], ignore_index=True)
        print(df)
# save the results to a csv file
df.to_csv('results_different_labwares.csv', index=False)


source_dim=12, dest_dim=12
repeat: 1
num_candidates=12
CVRP_distance: 25.77
row-major sorting: 30
LAP_distance: 28
greedy_optimized_distance: 32.85
num_candidates=24
CVRP_distance: 43.84
row-major sorting: 58
LAP_distance: 53
greedy_optimized_distance: 61.86
num_candidates=36
CVRP_distance: 69.3
row-major sorting: 94
LAP_distance: 87
greedy_optimized_distance: 94.79
num_candidates=48
CVRP_distance: 77.28
row-major sorting: 125
LAP_distance: 99
greedy_optimized_distance: 130.34
num_candidates=60
CVRP_distance: 96.21
row-major sorting: 146
LAP_distance: 129
greedy_optimized_distance: 166.11
num_candidates=72
CVRP_distance: 111.71
row-major sorting: 176
LAP_distance: 155
greedy_optimized_distance: 192.38
num_candidates=84
CVRP_distance: 128.94
row-major sorting: 201
LAP_distance: 186
greedy_optimized_distance: 222.6
num_candidates=96
CVRP_distance: 138.63
row-major sorting: 239
LAP_distance: 195
greedy_optimized_distance: 274.04
num_candidates=108
CVRP_distance: 154.84
row-major sorting: 

/var/folders/rt/qt1716456c913tnjd66ntj8w0000gn/T/ipykernel_79490/962504384.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, stats], ignore_index=True)


CVRP_distance: 23.87
row-major sorting: 26
LAP_distance: 25
greedy_optimized_distance: 28.32
num_candidates=24
CVRP_distance: 52.74
row-major sorting: 59
LAP_distance: 60
greedy_optimized_distance: 64.1
num_candidates=36
CVRP_distance: 68.77
row-major sorting: 96
LAP_distance: 86
greedy_optimized_distance: 105.2
num_candidates=48
CVRP_distance: 83.33
row-major sorting: 127
LAP_distance: 106
greedy_optimized_distance: 138.17
num_candidates=60
CVRP_distance: 95.5
row-major sorting: 147
LAP_distance: 127
greedy_optimized_distance: 155.23
num_candidates=72
CVRP_distance: 112.84
row-major sorting: 177
LAP_distance: 150
greedy_optimized_distance: 192.57
num_candidates=84
CVRP_distance: 124.33
row-major sorting: 210
LAP_distance: 166
greedy_optimized_distance: 223.26
num_candidates=96
CVRP_distance: 140.51
row-major sorting: 235
LAP_distance: 189
greedy_optimized_distance: 261.15
num_candidates=108
CVRP_distance: 149.52
row-major sorting: 249
LAP_distance: 200
greedy_optimized_distance: 280.5

In [ ]:
# plot the results
for labware_combination in labware_combinations:
    source = labware_combination[0]
    dest = labware_combination[1]
    df_choosen = df[(df['source_labware'] == source) & (df['dest_labware'] == dest)]
    # compute the mean and std of the unoptimized, rowwise, greedy and VRP
    df_choosen_mean = df_choosen.groupby(['num_samples']).mean().reset_index()
    df_choosen_std = df_choosen.groupby(['num_samples']).std().reset_index()
    # plot the results
    plt.figure(figsize=(4, 3),dpi=200,clear=True)
    plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['row-major sorting'], yerr=df_choosen_std['row-major sorting'], label='row-major sorting', fmt='o-', capsize=5)
    plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['LAP'], yerr=df_choosen_std['LAP'], label='LAP', fmt='o-', capsize=5)
    plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['greedy'], yerr=df_choosen_std['greedy'], label='greedy', fmt='o-', capsize=5)
    plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['CVRP'], yerr=df_choosen_std['CVRP'], label='CVRP', fmt='o-', capsize=5)
    plt.title(f' {source}-well plate --> {dest}-well plate', fontsize=15)
    plt.xlabel('Number of liquid transfers', fontsize=12)
    plt.ylabel('Computed execution time (s)', fontsize=12)
    # tick labels size
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.locator_params(nbins=7)
    plt.legend(frameon=False)
    #plt.tight_layout()
    plt.show()

In [ ]:
# optimization for 1536-well plates
np.random.seed(0)
for labware_combination in labware_combinations: 

    source_dim = 1536
    dest_dim = 1536
    print(f'source_dim={source_dim}, dest_dim={dest_dim}')
    # repeat the experiment 3 times
    for r in range(3):
        print('repeat:',r+1)
        stats = []
        for i in range(1, 11, 2):

            num_candidates = dest_dim * i 
            num_candidates = int(num_candidates)
            print(f'num_candidates={num_candidates}')
            a = random_task_generation(source_dim,dest_dim,num_candidates)
            jobs = np.argwhere(a)
            D_S = calculate_D(a.shape[0])
            D_D = calculate_D(a.shape[1])
            S, E, volumes = calculate_S_E(a)

            # calculate distance matrix for CVRP
            D_prime = calculate_D_prime(D_S, D_D, S, E, volumes, 1, 100, 1, 100)
            # VRP solver
            if num_candidates <7000:
                CVRP_distance, CVRP_recorder = CVRP_solver(np.round(D_prime * 100).astype(np.int64), solving_time=60)
            else:
                CVRP_distance, CVRP_recorder = CVRP_solver(np.round(D_prime * 100).astype(np.int64), solving_time=120)
            # bring the VRP_distance to the original scale
            CVRP_distance = CVRP_distance / 100
            print(f'CVRP_distance: {CVRP_distance}')

            # calculate the cost of the row-major sorting
            # create a sequence of tasks
            tasks = np.array(range(jobs.shape[0]))
            tasks = tasks+1
            # if tasks.shape[0] %8 != 0, pad with -1
            if tasks.shape[0] %8 != 0:
                tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
            rms_seuqnece = tasks.reshape(-1, 8)
            t = calculate_X(rms_seuqnece)
            d = np.round(D_prime * 100)
            rms_distance = np.trace(np.dot(t.T, d))/100
            rms_distance = int(rms_distance)
            print(f'row-major sorting: {rms_distance}')
            
            # calculate the cost of the LAP
            index_matrix = np.zeros((source_dim,dest_dim))
            for j in range(jobs.shape[0]):
                index_matrix[jobs[j, 0], jobs[j, 1]] = j+1
            LAP_sequence = LAP(index_matrix)
            if LAP_sequence.shape[0] %8 != 0:
                LAP_sequence = np.pad(LAP_sequence, (0, 8-LAP_sequence.shape[0]%8), 'constant', constant_values=-1)
            LAP_sequence = LAP_sequence.reshape(-1, 8)
            t = calculate_X(LAP_sequence)
            LAP_distance = np.trace(np.dot(t.T, d))/100
            # change non_optimized_distance to integer
            LAP_distance = int(LAP_distance)
            print(f'LAP_distance: {LAP_distance}')

            # calculate the cost of the greedy optimized sequence
            greedy_optimized_sequence = greedy(jobs, d[1:,1:])
            if greedy_optimized_sequence.shape[0] %8 != 0:
                greedy_optimized_sequence = np.pad(greedy_optimized_sequence, (0, 8-greedy_optimized_sequence.shape[0]%8), 'constant', constant_values=-1)
            greedy_optimized_sequence = greedy_optimized_sequence.reshape(-1, 8)
            t = calculate_X(greedy_optimized_sequence)
            greedy_optimized_distance = np.trace(np.dot(t.T, d))/100
            print(f'greedy_optimized_distance: {greedy_optimized_distance}')
            # append the results to the df
            stats.append([source_dim, dest_dim, num_candidates, r+1, rms_distance, LAP_distance, greedy_optimized_distance, CVRP_distance])
        # convert the stats to the dataframe
        stats = pd.DataFrame(stats, columns=['source_labware', 'dest_labware', 'num_samples', 'repeat', 'row-major sorting','LAP', 'greedy','CVRP'])
        # append the stats to the df
        df = pd.concat([df, stats], ignore_index=True)
        print(df)
# save the results to a csv file
df.to_csv('1536_results.csv', index=False)


In [ ]:
source = 1536
dest = 1536
df_choosen = df[(df['source_labware'] == source) & (df['dest_labware'] == dest)]
# compute the mean and std of the unoptimized, rowwise, greedy and VRP
df_choosen_mean = df_choosen.groupby(['num_samples']).mean().reset_index()
df_choosen_std = df_choosen.groupby(['num_samples']).std().reset_index()
# plot the results
plt.figure(figsize=(4, 3),dpi=200,clear=True)
plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['row-major sorting'], yerr=df_choosen_std['row-major sorting'], label='row-major sorting', fmt='o-', capsize=5)
plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['LAP'], yerr=df_choosen_std['LAP'], label='LAP', fmt='o-', capsize=5)
plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['greedy'], yerr=df_choosen_std['greedy'], label='greedy', fmt='o-', capsize=5)
plt.errorbar(df_choosen_mean['num_samples'], df_choosen_mean['CVRP'], yerr=df_choosen_std['CVRP'], label='CVRP', fmt='o-', capsize=5)
plt.title(f' {source}-well plate --> {dest}-well plate', fontsize=15)
plt.xlabel('Number of liquid transfers', fontsize=12)
plt.ylabel('Computed execution time (s)', fontsize=12)
# tick labels size
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.locator_params(nbins=7)
plt.legend(frameon=False)
#plt.tight_layout()
plt.show()